In [1]:
import copy
import requests
import numpy as np
import pandas as pd
import csv
import os
import time
import subprocess

In [2]:
def Price_update():
    # Load both CSV files
    investment_df = pd.read_csv("Stock_final_investment.csv")
    prices_df = pd.read_csv("stock_prices.csv")

    # Merge and extract 'Last Price ($)' column
    merged_df = pd.merge(
        investment_df[['Sector', 'Stock']],
        prices_df[['Sector', 'Stock', 'Last Price ($)']],
        on=['Sector', 'Stock'],
        how='left'
    )   
    # Return just the 'Last Price ($)' column
    last_prices = merged_df['Last Price ($)']
    return last_prices

In [3]:
wallet = pd.read_csv("Money.csv")["Wallet"].iloc[0]

#### Variables for New Stocks

In [4]:
stocks = pd.read_csv("Stock_final_investment.csv")
stocks_name = stocks["Stock"]
stock_old_price = stocks["Price"]
stock_buy_price = copy.deepcopy(stock_old_price)
stock_weights = stocks["Weights"]
returns = pd.read_csv("Money.csv")["Returns"].iloc[0]
stock_quant=[0 for i in range(10)]

#### Calculating Quantity of each stock to purchase

In [5]:
if (pd.read_csv("old_stock_data.csv")["Buy Price"]).empty:
    stock_quant = (wallet * (stock_weights / 100)) // stock_buy_price
else:
    stock_quant = ((10000+wallet) * (stock_weights / 100)) // stock_buy_price
best_choice=stock_buy_price.iloc[0]
best_choice_index=0
current_wallet=0
for i in range(len(stocks_name)):
    stock_quant[i]=int(stock_quant[i])
    current_wallet+=stock_quant[i]*stock_buy_price.iloc[i]
for i in range(len(stocks_name)//2):
    if(stock_buy_price.iloc[i]<best_choice):
        best_choice=stock_buy_price.iloc[i]
        best_price_index=i

current_wallet=wallet-current_wallet
if(current_wallet//best_choice>0):
    stock_quant[best_choice_index]+=int(current_wallet//best_choice)

current_wallet=0
for i in range(len(stocks_name)):
    stock_quant[i]=int(stock_quant[i])
    current_wallet+=stock_quant[i]*stock_buy_price.iloc[i]
for i in range(best_choice_index,len(stocks_name)):
    if(stock_buy_price.iloc[i]<best_choice):
        best_choice=stock_buy_price.iloc[i]
        best_price_index=i

current_wallet=wallet-current_wallet
if(current_wallet//best_choice>0):
    stock_quant[best_choice_index]+=int(current_wallet//best_choice)

current_wallet-=(current_wallet//best_choice)*best_choice
stock_quant

0     72.0
1     16.0
2     69.0
3    344.0
4     11.0
5      7.0
6     13.0
7      8.0
8     11.0
9      7.0
dtype: float64

#### Variables for Old Stocks

In [6]:
stocks_old = pd.read_csv("old_stock_data.csv")
stocks_name_old = stocks_old["Stock"]
stock_buy_price_old = stocks_old["Buy Price"]
stock_quant_old = stocks_old["Quantity"]
stock_last_price = stocks_old["Last Price"]

#### Removing capital from stocks which aren't in new chosen stocks

In [7]:
rp = np.zeros(len(stock_buy_price))  # match size with number of stocks
map=[0 for i in range(10)]
for i in range(len(stocks_name_old)):
    for j in range(len(stocks_name)):
        if stocks_name_old.iloc[i] == stocks_name.iloc[j]:
            diff = stock_quant_old.iloc[i] - stock_quant.iloc[j]
            if diff > 0:
                returns += diff * (stock_last_price.iloc[i] - stock_buy_price.iloc[j])
                wallet += diff * stock_last_price.iloc[i]
            elif diff < 0:
                wallet -= diff * stock_buy_price.iloc[j]
            map[i]=1
    for j in range(len(stocks_name)):
        if(map[j]!=1):
            wallet+=stock_last_price.iloc[j]*stock_quant_old.iloc[j]

#### Investing and withdrawing again and again

In [8]:
# Control flag
f = 1
start_time = time.time()  # Start timer

while f == 1:
    subprocess.run(["jupyter", "nbconvert", "--execute", "--to", "notebook", "--inplace", "Stock_universe.ipynb"])
    stock_new_price = Price_update()
    diff_price = stock_new_price - stock_old_price

    for i in range(len(stock_new_price)):
        if diff_price.iloc[i] < 0:
            wallet += stock_new_price.iloc[i] * stock_quant[i]
            returns += (stock_new_price.iloc[i]-stock_old_price.iloc[i]) * stock_quant[i]
            rp[i] += stock_new_price.iloc[i] * stock_quant[i]
            stock_quant[i] = 0

        elif (diff_price.iloc[i] > 0) and (wallet > stock_new_price.iloc[i]):
            quantity = int(wallet*stock_weights.iloc[i]/ stock_new_price.iloc[i])
            wallet -= quantity * stock_new_price.iloc[i]
            stock_quant[i] += quantity
            stock_buy_price[i] = stock_new_price.iloc[i]

        if wallet>0:
            quantity = int(wallet/stock_new_price.iloc[best_choice_index])
            wallet -= quantity * stock_new_price.iloc[best_choice_index]
            stock_quant[best_choice_index] += quantity
            stock_buy_price[best_choice_index] = stock_new_price.iloc[best_choice_index]


    stock_old_price = stock_new_price.copy()
    # Check if 45 minutes passed
    # if (time.time() - start_time) >= 60 * 30:
        # print("30 minutes passed — ending loop.")
    f = 0

#### Storing Data

In [9]:
stocks_data_old={
    "Stock":stocks_name,
    "Quantity":stock_quant,
    "Buy Price":stock_buy_price,
    "Last Price":Price_update()
}
df=pd.DataFrame(stocks_data_old)
df.to_csv("old_stock_data.csv",index=False)

In [10]:
money = {
    "Wallet": [wallet],
    "Returns": [returns]
}
dframe = pd.DataFrame(money)
dframe.to_csv("Money.csv", index=False)


In [11]:
file_path = "return_statements.csv"
file_exists = os.path.isfile(file_path) and os.path.getsize(file_path) > 0

with open(file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(["Returns"])  # Write header only once
    writer.writerow([returns])  # Write the actual return value

#### Fetching Yield Percent and Calculating Sharpe Ratio and Total Returns

In [12]:
# Replace with your FRED API Key
API_KEY = "d19ebaf43edd81021d47f34677bbfc3f"

# 10-Year Treasury Yield (You can change the series ID for other maturities)
URL = f"https://api.stlouisfed.org/fred/series/observations?series_id=DGS10&api_key={API_KEY}&file_type=json"

response = requests.get(URL)

if response.status_code == 200:
    data = response.json()
    latest_yield = data['observations'][-1]['value']  # Last available value
    print(f"Latest 10-Year Treasury Yield: {latest_yield}%")
else:
    print("Error fetching data")

Latest 10-Year Treasury Yield: 4.06%


In [13]:
return_prev=pd.read_csv("return_statements.csv")["Returns"]
std_dev = np.std(return_prev)
if std_dev != 0:
    Sharpe_Ratio = (returns/100 - float(latest_yield)) / std_dev
else:
    Sharpe_Ratio = 0  # or float('nan') or some default value
print(f"Sharpe Ratio = {Sharpe_Ratio}\n")
print(f"Today's Returns = {returns}$\n")
total_returns=sum(return_prev)
print(f"Total Returns = {total_returns}$\n")

profit={
    "Sharpe Ratio":[Sharpe_Ratio],
    "Today's Returns":[returns],
    "Total Returns":[total_returns]
}
dtframe=pd.DataFrame(profit)
dtframe.to_csv("Net_profit.csv",index=False)

Sharpe Ratio = 0.028958469365802198

Today's Returns = 15965.100000000002$

Total Returns = 36497.65500000001$

